In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns

import suppression as s

In [7]:
gaba_fn = 'gaba_data.txt'
supp_fn = 'supp_data_individual_20170411.txt'

In [8]:
sdf = s.load_psychophys(supp_fn)
gdf = s.load_gaba(gaba_fn)

In [9]:
sdf[(sdf['Task']=='SS') & (sdf['Subject']=='ai')]

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev
0,ai,SS,nMono,Iso,Nde,Control,10.0,0.615,0.009,1.449,Control-Nde,6.900,-0.211125
1,ai,SS,nMono,Iso,Nde,Control,30.0,0.671,0.010,4.348,Control-Nde,6.900,-0.173277
2,ai,SS,nMono,Iso,Nde,Control,62.0,1.658,0.007,8.986,Control-Nde,6.900,0.219585
139,ai,SS,nMono,Iso,De,Control,10.0,0.588,0.005,1.440,Control-De,6.943,-0.230623
140,ai,SS,nMono,Iso,De,Control,30.0,1.032,0.006,4.321,Control-De,6.943,0.013680
141,ai,SS,nMono,Iso,De,Control,62.0,1.719,0.005,8.930,Control-De,6.943,0.235276
246,ai,SS,nMono,Cross,Nde,Control,10.0,0.480,0.008,1.449,Control-Nde,6.900,-0.318759
247,ai,SS,nMono,Cross,Nde,Control,30.0,0.698,0.008,4.348,Control-Nde,6.900,-0.156145
248,ai,SS,nMono,Cross,Nde,Control,62.0,0.911,0.009,8.986,Control-Nde,6.900,-0.040482
386,ai,SS,nMono,Cross,De,Control,10.0,0.447,0.008,1.440,Control-De,6.943,-0.349692


In [10]:
10.0/6.900

1.4492753623188406

In [11]:
gdf.head()

,subjName,Population,Presentation,GABA
0,ac,Amblyope,occ_binoc,0.196
1,ac,Amblyope,occ_none,0.203
2,am,Amblyope,occ_binoc,0.193
3,am,Amblyope,occ_none,0.188
4,as,Amblyope,occ_binoc,0.217


## Identify common subjects, subset data frames to only include these ##

In [12]:
pp_subjs = np.unique(sdf.Subject)
gaba_subjs = np.unique(gdf.subjName)
use_subjs = list(np.intersect1d(pp_subjs, gaba_subjs))
print(pp_subjs, len(pp_subjs))
print(gaba_subjs, len(gaba_subjs))
print(use_subjs, len(use_subjs))

['ah' 'ai' 'am' 'ap' 'bi' 'bo' 'cl' 'cr' 'ct' 'dl' 'ds' 'eg' 'ei' 'em' 'es'
 'gd' 'gm' 'gw' 'jd' 'jv' 'ke' 'kw' 'li' 'ls' 'mg' 'mk' 'ms' 'mv' 'nl' 'nn'
 'rn' 'sd' 'se' 'tj' 'tm' 'tt' 'yl'] 37
['ac' 'ai' 'am' 'as' 'bi' 'ch' 'cr' 'cs' 'ct' 'dl' 'ei' 'em' 'es' 'gd' 'gm'
 'jd' 'jm' 'jv' 'ks' 'kw' 'li' 'ls' 'mg' 'mk' 'ms' 'nl' 'ps' 'rn' 'sd' 'tt'
 'yl'] 31
['ai', 'am', 'bi', 'cr', 'ct', 'dl', 'ei', 'em', 'es', 'gd', 'gm', 'jd', 'jv', 'kw', 'li', 'ls', 'mg', 'mk', 'ms', 'nl', 'rn', 'sd', 'tt', 'yl'] 24


In [13]:
gdf = gdf[gdf.subjName.isin(use_subjs)] # only subjects who did both tasks
gdf = gdf[gdf.Presentation=='occ_binoc'] # use this GABA measure
gdf

,subjName,Population,Presentation,GABA
2,am,Amblyope,occ_binoc,0.193
6,bi,Amblyope,occ_binoc,0.207
12,em,Amblyope,occ_binoc,0.182
14,gd,Amblyope,occ_binoc,0.203
16,jv,Amblyope,occ_binoc,0.215
20,li,Amblyope,occ_binoc,0.227
22,mg,Amblyope,occ_binoc,0.210
26,rn,Amblyope,occ_binoc,0.202
28,sd,Amblyope,occ_binoc,0.190
30,tt,Amblyope,occ_binoc,0.179


In [14]:
len(gdf)

24

In [15]:
sdf = sdf[sdf.Subject.isin(use_subjs)] # only subjects who did both tasks
# make sure there are only as many unique subjNames in sdf as there are entries in gdf.subjName
assert(len(np.unique(sdf.Subject))==len(gdf.subjName))

## Model Psychophysical data ##
### ThreshElev as a function of logRelContrast ###

In [34]:
pp_gvars = ['Task','Orientation','Presentation','Population','Subject','Eye','Trace']
pp_gvars_base = pp_gvars + ['BaselineThresh']

In [17]:
print(pp_gvars, '\n', pp_gvars_base)

['Task', 'Orientation', 'Presentation', 'Population', 'Subject', 'Eye', 'Trace'] 
 ['Task', 'Orientation', 'Presentation', 'Population', 'Subject', 'Eye', 'Trace', 'BaselineThresh']


In [18]:
# create groups based on these grouping variables
groups = sdf.groupby(pp_gvars)
print(len(groups))
for g in groups:
    print(g)

364
(('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De'),      Subject Task Presentation Orientation Eye Population  MaskContrast  \
1887      am   OS       nDicho       Cross  De   Amblyope          11.0   
1888      am   OS       nDicho       Cross  De   Amblyope          16.0   
1889      am   OS       nDicho       Cross  De   Amblyope          23.0   
1890      am   OS       nDicho       Cross  De   Amblyope          32.0   
1891      am   OS       nDicho       Cross  De   Amblyope          45.0   

      ThreshElev  ThreshElev_SE  RelMaskContrast        Trace  BaselineThresh  \
1887       1.011          0.010            4.120  Amblyope-De           2.184   
1888       1.139          0.006            5.992  Amblyope-De           2.184   
1889       0.749          0.013            8.614  Amblyope-De           2.184   
1890       1.302          0.007           11.984  Amblyope-De           2.184   
1891       1.128          0.006           16.853  Amblyope-De           2

In [19]:
goi = groups.get_group(('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De'))

In [20]:
goi

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev
1892,bi,OS,nDicho,Cross,De,Amblyope,5.0,1.024,0.010,1.848,Amblyope-De,1.668,0.010300
1893,bi,OS,nDicho,Cross,De,Amblyope,10.0,0.906,0.007,3.696,Amblyope-De,1.668,-0.042872
1894,bi,OS,nDicho,Cross,De,Amblyope,30.0,1.182,0.007,11.087,Amblyope-De,1.668,0.072617


In [21]:
goi2 = groups.get_group(('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde'))

In [22]:
goi2

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev
1790,bi,OS,nDicho,Cross,Nde,Amblyope,5.0,2.178,0.019,2.997,Amblyope-Nde,2.706,0.338058
1791,bi,OS,nDicho,Cross,Nde,Amblyope,10.0,17.232,0.027,5.994,Amblyope-Nde,2.706,1.236336
1792,bi,OS,nDicho,Cross,Nde,Amblyope,30.0,18.287,0.024,17.983,Amblyope-Nde,2.706,1.262142


### Figure out how do to t-tests between corresponding points (to find where the lines are most different)

In [36]:
pp_gvars_base

['Task',
 'Orientation',
 'Presentation',
 'Population',
 'Subject',
 'Eye',
 'Trace',
 'BaselineThresh']

In [48]:
gvars_ttest = ['Task','Orientation','Presentation','Population','Subject']
grouped_botheyes = sdf.groupby(gvars_ttest)
tt_onesub = grouped_botheyes.get_group(('OS', 'Cross', 'nDicho', 'Amblyope', 'bi'))
tt_onesub

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev
1790,bi,OS,nDicho,Cross,Nde,Amblyope,5.0,2.178,0.019,2.997,Amblyope-Nde,2.706,0.338058
1791,bi,OS,nDicho,Cross,Nde,Amblyope,10.0,17.232,0.027,5.994,Amblyope-Nde,2.706,1.236336
1792,bi,OS,nDicho,Cross,Nde,Amblyope,30.0,18.287,0.024,17.983,Amblyope-Nde,2.706,1.262142
1892,bi,OS,nDicho,Cross,De,Amblyope,5.0,1.024,0.010,1.848,Amblyope-De,1.668,0.010300
1893,bi,OS,nDicho,Cross,De,Amblyope,10.0,0.906,0.007,3.696,Amblyope-De,1.668,-0.042872
1894,bi,OS,nDicho,Cross,De,Amblyope,30.0,1.182,0.007,11.087,Amblyope-De,1.668,0.072617


In [50]:
tt_results = tt_onesub.groupby('MaskContrast').apply(np.mean)
tt_results

,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,BaselineThresh,logThreshElev
MaskContrast,,,,,,
5.0,5.0,1.6010,0.0145,2.4225,2.187,0.174179
10.0,10.0,9.0690,0.0170,4.8450,2.187,0.596732
30.0,30.0,9.7345,0.0155,14.5350,2.187,0.667380


### Linear model (defined in suppression.py, uses lmfit)###

In [35]:
# initialize parameters
lm_params = s.lf.Parameters()
lm_params.add('y_int', value=1)
lm_params.add('slope', value=1)
lm_params.keys()

odict_keys(['y_int', 'slope'])

### Model all data points, incl. facilitation (good for display)

In [36]:
# model each group's data using functions defined in suppression.py
preds_wfac = groups.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh, lm_params,
                     ret='preds', supp_only=False, log=False)

('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De')
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.01333     -inf      inf  0.02213     True     None
y_int     0.939     -inf      inf   0.2331     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'Nde', 'Amblyope-Nde')
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.01771     -inf      inf  0.03058     True     None
y_int     1.049     -inf      inf   0.3938     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De')
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.02288     -inf      inf  0.01669     True     None
y_int    0.9105     -inf      inf    0.114     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde')
Name      Value      Min      Max   Stderr     Vary     Expr
slope    0.7929     -inf      inf    0.814     True     None
y_int     5.436     -inf      inf    9.019     True     None
('OS

In [37]:
s.group_facet_plots(preds_wfac, s.fit_plot, 'plots/individual_linear_wfac_logaxes.pdf',
                  ['Subject','Task'], #each combo of this gets its own page
                  row='Orientation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Presentation",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshElev") 

Plotting ai.OS
Plotting ai.SS
Plotting am.OS
Plotting am.SS
Plotting bi.OS
Plotting bi.SS
Plotting cr.OS
Plotting cr.SS
Plotting ct.OS
Plotting ct.SS
Plotting dl.OS
Plotting dl.SS
Plotting ei.OS
Plotting ei.SS
Plotting em.SS
Plotting es.OS
Plotting es.SS
Plotting gd.OS
Plotting gd.SS
Plotting gm.OS
Plotting gm.SS


//anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Plotting jd.OS
Plotting jd.SS
Plotting jv.OS
Plotting jv.SS
Plotting kw.OS
Plotting kw.SS
Plotting li.OS
Plotting li.SS
Plotting ls.OS
Plotting ls.SS
Plotting mg.OS
Plotting mg.SS
Plotting mk.OS
Plotting mk.SS
Plotting ms.OS
Plotting ms.SS
Plotting nl.OS
Plotting nl.SS
Plotting rn.OS
Plotting rn.SS
Plotting sd.OS
Plotting sd.SS
Plotting tt.OS
Plotting yl.OS
Plotting yl.SS
Plots saved at plots/individual_linear_wfac_logaxes.pdf


In [38]:
s.group_facet_plots(preds_wfac, s.fit_plot, 'plots/individual_linear_wfac_logaxes_withpred.pdf',
                  ['Subject','Task'], #each combo of this gets its own page
                  row='Orientation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Presentation",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshPred")

Plotting ai.OS
Plotting ai.SS
Plotting am.OS
Plotting am.SS
Plotting bi.OS
Plotting bi.SS
Plotting cr.OS
Plotting cr.SS
Plotting ct.OS
Plotting ct.SS
Plotting dl.OS
Plotting dl.SS
Plotting ei.OS
Plotting ei.SS
Plotting em.SS
Plotting es.OS
Plotting es.SS
Plotting gd.OS
Plotting gd.SS
Plotting gm.OS
Plotting gm.SS


//anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Plotting jd.OS
Plotting jd.SS
Plotting jv.OS
Plotting jv.SS
Plotting kw.OS
Plotting kw.SS
Plotting li.OS
Plotting li.SS
Plotting ls.OS
Plotting ls.SS
Plotting mg.OS
Plotting mg.SS
Plotting mk.OS
Plotting mk.SS
Plotting ms.OS
Plotting ms.SS
Plotting nl.OS
Plotting nl.SS
Plotting rn.OS
Plotting rn.SS
Plotting sd.OS
Plotting sd.SS
Plotting tt.OS
Plotting yl.OS
Plotting yl.SS
Plots saved at plots/individual_linear_wfac_logaxes_withpred.pdf


In [39]:
s.group_facet_plots(preds_wfac, s.fit_plot, 'plots/combined_linear_wfac_logaxes_nopred.pdf',
                  ['Task','Orientation'], #each combo of this gets its own page
                  row='Presentation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Subject",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshElev") 

Plotting OS.Cross
Plotting OS.Iso
Plotting SS.Cross
Plotting SS.Iso
Plots saved at plots/combined_linear_wfac_logaxes_nopred.pdf


### Now model only the non-facilitation points

In [26]:
# model each group's data using functions defined in suppression.py
preds = groups.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh, lm_params,
                     ret='preds', supp_only=True, log=False)

('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De')
Name      Value      Min      Max   Stderr     Vary     Expr
slope  -0.03574     -inf      inf      inf     True     None
y_int      1.73     -inf      inf      inf     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'Nde', 'Amblyope-Nde')
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.05783     -inf      inf   0.0317     True     None
y_int    0.3931     -inf      inf   0.5013     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De')
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde')
Name      Value      Min      Max   Stderr     Vary     Expr
slope    0.7929     -inf      inf    0.814     True     None
y_int     5.436     -inf      inf    9.019     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'gd', 'De', 'Amblyope-De')
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.07379     -inf      inf 0.001199     True     None


In [27]:
preds[preds['Subject']=='bi']

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev,ThreshPred
95,bi,SS,nMono,Iso,Nde,Amblyope,10.0,0.838,0.008,0.986,Amblyope-Nde,10.141,-0.076756,NaN
96,bi,SS,nMono,Iso,Nde,Amblyope,30.0,1.195,0.008,2.958,Amblyope-Nde,10.141,0.077368,1.195000
97,bi,SS,nMono,Iso,Nde,Amblyope,50.0,1.816,0.011,4.931,Amblyope-Nde,10.141,0.259116,1.816000
203,bi,SS,nMono,Iso,De,Amblyope,10.0,0.837,0.006,1.278,Amblyope-De,7.827,-0.077275,NaN
204,bi,SS,nMono,Iso,De,Amblyope,30.0,1.623,0.006,3.833,Amblyope-De,7.827,0.210319,1.623000
205,bi,SS,nMono,Iso,De,Amblyope,50.0,2.350,0.006,6.389,Amblyope-De,7.827,0.371068,2.350000
343,bi,SS,nMono,Cross,Nde,Amblyope,10.0,1.000,0.009,0.986,Amblyope-Nde,10.141,0.000000,NaN
344,bi,SS,nMono,Cross,Nde,Amblyope,30.0,1.070,0.010,2.958,Amblyope-Nde,10.141,0.029384,1.070000
345,bi,SS,nMono,Cross,Nde,Amblyope,50.0,1.265,0.009,4.931,Amblyope-Nde,10.141,0.102091,1.265000
449,bi,SS,nMono,Cross,De,Amblyope,10.0,0.931,0.008,1.278,Amblyope-De,7.827,-0.031050,NaN


In [33]:
s.group_facet_plots(preds, s.fit_plot, 'plots/individual_linear_logaxes.pdf',
                  ['Subject','Task'], #each combo of this gets its own page
                  row='Orientation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Presentation",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshElev") 

Plotting ai.OS
Plotting ai.SS
Plotting am.OS
Plotting am.SS
Plotting bi.OS
Plotting bi.SS
Plotting cr.OS
Plotting cr.SS
Plotting ct.OS
Plotting ct.SS
Plotting dl.OS
Plotting dl.SS
Plotting ei.OS
Plotting ei.SS
Plotting em.SS
Plotting es.OS
Plotting es.SS
Plotting gd.OS
Plotting gd.SS
Plotting gm.OS
Plotting gm.SS


//anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Plotting jd.OS
Plotting jd.SS
Plotting jv.OS
Plotting jv.SS
Plotting kw.OS
Plotting kw.SS
Plotting li.OS
Plotting li.SS
Plotting ls.OS
Plotting ls.SS
Plotting mg.OS
Plotting mg.SS
Plotting mk.OS
Plotting mk.SS
Plotting ms.OS
Plotting ms.SS
Plotting nl.OS
Plotting nl.SS
Plotting rn.OS
Plotting rn.SS
Plotting sd.OS
Plotting sd.SS
Plotting tt.OS
Plotting yl.OS
Plotting yl.SS
Plots saved at plots/individual_linear_logaxes.pdf


In [32]:
s.group_facet_plots(preds, s.fit_plot, 'plots/individual_linear_logaxes_withpred.pdf',
                  ['Subject','Task'], #each combo of this gets its own page
                  row='Orientation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Presentation",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshPred") 

Plotting ai.OS
Plotting ai.SS
Plotting am.OS
Plotting am.SS
Plotting bi.OS
Plotting bi.SS
Plotting cr.OS
Plotting cr.SS
Plotting ct.OS
Plotting ct.SS
Plotting dl.OS
Plotting dl.SS
Plotting ei.OS
Plotting ei.SS
Plotting em.SS
Plotting es.OS
Plotting es.SS
Plotting gd.OS
Plotting gd.SS
Plotting gm.OS
Plotting gm.SS


//anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Plotting jd.OS
Plotting jd.SS
Plotting jv.OS
Plotting jv.SS
Plotting kw.OS
Plotting kw.SS
Plotting li.OS
Plotting li.SS
Plotting ls.OS
Plotting ls.SS
Plotting mg.OS
Plotting mg.SS
Plotting mk.OS
Plotting mk.SS
Plotting ms.OS
Plotting ms.SS
Plotting nl.OS
Plotting nl.SS
Plotting rn.OS
Plotting rn.SS
Plotting sd.OS
Plotting sd.SS
Plotting tt.OS
Plotting yl.OS
Plotting yl.SS
Plots saved at plots/individual_linear_logaxes_withpred.pdf


In [29]:
s.group_facet_plots(preds, s.fit_plot, 'plots/combined_linear_logaxes_nopred.pdf',
                  ['Task','Orientation'], #each combo of this gets its own page
                  row='Presentation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Subject",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshElev") 

Plotting OS.Cross
Plotting OS.Iso
Plotting SS.Cross
Plotting SS.Iso
Plots saved at plots/combined_linear_logaxes_nopred.pdf


In [30]:
s.group_facet_plots(preds, s.fit_plot, 'plots/combined_linear_logaxes_withpred.pdf',
                  ['Task','Orientation'], #each combo of this gets its own page
                  row='Presentation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="ThreshElev", # x, y
                  hue="Subject",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='x--',Ycol="ThreshPred") 

Plotting OS.Cross
Plotting OS.Iso
Plotting SS.Cross
Plotting SS.Iso
Plots saved at plots/combined_linear_logaxes_withpred.pdf


### Model the data again, but this time return parameters, not predictions ###

In [48]:
# suppression only (no facilitation data points)
# model each group's data using functions defined in suppression.py, return params
groups_with_baseline = sdf.groupby(pp_gvars_base)
pfit = groups_with_baseline.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh, lm_params, ret='weights', supp_only=True).reset_index()

('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De', 2.4859999999999998)
Name      Value      Min      Max   Stderr     Vary     Expr
slope  -0.03574     -inf      inf      inf     True     None
y_int      1.73     -inf      inf      inf     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'Nde', 'Amblyope-Nde', 2.7539999999999996)
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.05783     -inf      inf   0.0317     True     None
y_int    0.3931     -inf      inf   0.5013     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De', 1.76)
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde', 1.9159999999999999)
Name      Value      Min      Max   Stderr     Vary     Expr
slope    0.7929     -inf      inf    0.814     True     None
y_int     5.436     -inf      inf    9.019     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'gd', 'De', 'Amblyope-De', 2.0709999999999997)
Name      Value      Min      Max   

In [67]:
# include facilitation data points
# model each group's data using functions defined in suppression.py, return params
groups_with_baseline = sdf.groupby(pp_gvars_base)
pfit = groups_with_baseline.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh, lm_params, ret='weights', supp_only=False).reset_index()

('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De', 2.4859999999999998)
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.01333     -inf      inf  0.02213     True     None
y_int     0.939     -inf      inf   0.2331     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'Nde', 'Amblyope-Nde', 2.7539999999999996)
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.01771     -inf      inf  0.03058     True     None
y_int     1.049     -inf      inf   0.3938     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De', 1.76)
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.02288     -inf      inf  0.01669     True     None
y_int    0.9105     -inf      inf    0.114     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde', 1.9159999999999999)
Name      Value      Min      Max   Stderr     Vary     Expr
slope    0.7929     -inf      inf    0.814     True     None

In [69]:
# including fac
pfit

,Task,Orientation,Presentation,Population,Subject,Eye,Trace,BaselineThresh,y_int,slope
0,OS,Cross,nDicho,Amblyope,am,De,Amblyope-De,2.486,0.939042,0.013325
1,OS,Cross,nDicho,Amblyope,am,Nde,Amblyope-Nde,2.754,1.049378,0.017715
2,OS,Cross,nDicho,Amblyope,bi,De,Amblyope-De,1.760,0.910471,0.022884
3,OS,Cross,nDicho,Amblyope,bi,Nde,Amblyope-Nde,1.916,5.436084,0.792939
4,OS,Cross,nDicho,Amblyope,gd,De,Amblyope-De,2.071,0.650107,0.067567
5,OS,Cross,nDicho,Amblyope,gd,Nde,Amblyope-Nde,2.112,0.072075,0.394327
6,OS,Cross,nDicho,Amblyope,jv,De,Amblyope-De,1.777,0.644637,0.060397
7,OS,Cross,nDicho,Amblyope,jv,Nde,Amblyope-Nde,2.482,1.727919,0.105221
8,OS,Cross,nDicho,Amblyope,li,De,Amblyope-De,2.238,1.369412,-0.025260
9,OS,Cross,nDicho,Amblyope,li,Nde,Amblyope-Nde,4.555,4.121034,0.093321


In [50]:
pfit

,Task,Orientation,Presentation,Population,Subject,Eye,Trace,BaselineThresh,y_int,slope
0,OS,Cross,nDicho,Amblyope,am,De,Amblyope-De,2.486,1.730264,-0.035736
1,OS,Cross,nDicho,Amblyope,am,Nde,Amblyope-Nde,2.754,0.393058,0.057829
2,OS,Cross,nDicho,Amblyope,bi,De,Amblyope-De,1.760,NaN,NaN
3,OS,Cross,nDicho,Amblyope,bi,Nde,Amblyope-Nde,1.916,5.436084,0.792939
4,OS,Cross,nDicho,Amblyope,gd,De,Amblyope-De,2.071,0.533715,0.073787
5,OS,Cross,nDicho,Amblyope,gd,Nde,Amblyope-Nde,2.112,0.072075,0.394327
6,OS,Cross,nDicho,Amblyope,jv,De,Amblyope-De,1.777,NaN,NaN
7,OS,Cross,nDicho,Amblyope,jv,Nde,Amblyope-Nde,2.482,1.727919,0.105221
8,OS,Cross,nDicho,Amblyope,li,De,Amblyope-De,2.238,NaN,NaN
9,OS,Cross,nDicho,Amblyope,li,Nde,Amblyope-Nde,4.555,4.121034,0.093321


In [70]:
pp_gvars_base

['Task',
 'Orientation',
 'Presentation',
 'Population',
 'Subject',
 'Eye',
 'Trace',
 'BaselineThresh']

In [71]:
# melt the result of the modeling into long format for plotting
m = pd.melt(pfit, id_vars=pp_gvars, var_name='measure')
m

,Task,Orientation,Presentation,Population,Subject,Eye,Trace,measure,value
0,OS,Cross,nDicho,Amblyope,am,De,Amblyope-De,BaselineThresh,2.486000
1,OS,Cross,nDicho,Amblyope,am,Nde,Amblyope-Nde,BaselineThresh,2.754000
2,OS,Cross,nDicho,Amblyope,bi,De,Amblyope-De,BaselineThresh,1.760000
3,OS,Cross,nDicho,Amblyope,bi,Nde,Amblyope-Nde,BaselineThresh,1.916000
4,OS,Cross,nDicho,Amblyope,gd,De,Amblyope-De,BaselineThresh,2.071000
5,OS,Cross,nDicho,Amblyope,gd,Nde,Amblyope-Nde,BaselineThresh,2.112000
6,OS,Cross,nDicho,Amblyope,jv,De,Amblyope-De,BaselineThresh,1.777000
7,OS,Cross,nDicho,Amblyope,jv,Nde,Amblyope-Nde,BaselineThresh,2.482000
8,OS,Cross,nDicho,Amblyope,li,De,Amblyope-De,BaselineThresh,2.238000
9,OS,Cross,nDicho,Amblyope,li,Nde,Amblyope-Nde,BaselineThresh,4.555000


In [72]:
#Grab the GABA measure for each subject and it to each observation for easy plotting
comb = m.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [73]:
comb[(comb['Task']=='SS') & (comb['Orientation']=='Cross')]

,Task,Orientation,Presentation,Population,Subject,Eye,Trace,measure,value,GABA
180,SS,Cross,nDicho,Amblyope,am,De,Amblyope-De,BaselineThresh,7.532000,0.193
181,SS,Cross,nDicho,Amblyope,am,Nde,Amblyope-Nde,BaselineThresh,7.834000,0.193
182,SS,Cross,nDicho,Amblyope,bi,De,Amblyope-De,BaselineThresh,7.827000,0.207
183,SS,Cross,nDicho,Amblyope,bi,Nde,Amblyope-Nde,BaselineThresh,10.141000,0.207
184,SS,Cross,nDicho,Amblyope,em,De,Amblyope-De,BaselineThresh,5.949000,0.182
185,SS,Cross,nDicho,Amblyope,em,Nde,Amblyope-Nde,BaselineThresh,6.014000,0.182
186,SS,Cross,nDicho,Amblyope,gd,De,Amblyope-De,BaselineThresh,6.210000,0.203
187,SS,Cross,nDicho,Amblyope,gd,Nde,Amblyope-Nde,BaselineThresh,6.536000,0.203
188,SS,Cross,nDicho,Amblyope,jv,De,Amblyope-De,BaselineThresh,4.227000,0.215
189,SS,Cross,nDicho,Amblyope,jv,Nde,Amblyope-Nde,BaselineThresh,4.764000,0.215


In [75]:
#graphs!
with s.PdfPages('plots/gaba_vs_measures_wfac_linear_measure.pdf') as pdf:
    plot_groups = comb.groupby(['Task','Orientation','measure'])
    for gv, gr in plot_groups:
        print(gv, np.all(np.isnan(gr['value'])))
        if gv[0]=='SS' and gv[1]=='Cross': continue
        xvar = "GABA"
        yvar = "value"
        g = sns.lmplot(data=gr, 
                  row='Presentation',col='Population',# facet rows and columns
                  x=xvar, y=yvar,hue="Eye",sharey=False)
        g.fig.suptitle(gv, fontsize=16, y=0.97)
        g.fig.subplots_adjust(top=.9, right=.8)
        g.set_axis_labels(xvar, yvar)
        pdf.savefig(g.fig)
    plt.close('all')

('OS', 'Cross', 'BaselineThresh') False
('OS', 'Cross', 'slope') False
('OS', 'Cross', 'y_int') False
('OS', 'Iso', 'BaselineThresh') False
('OS', 'Iso', 'slope') False
('OS', 'Iso', 'y_int') False
('SS', 'Cross', 'BaselineThresh') False
('SS', 'Cross', 'slope') False
('SS', 'Cross', 'y_int') False
('SS', 'Iso', 'BaselineThresh') False
('SS', 'Iso', 'slope') False
('SS', 'Iso', 'y_int') False


In [76]:
#graphs!
with s.PdfPages('plots/gaba_vs_measures_wfac_linear_eye.pdf') as pdf:
    plot_groups = comb.groupby(['Task','Orientation','Eye'])
    for gv, gr in plot_groups:
        print(gv, np.all(np.isnan(gr['value'])))
        if gv[0]=='SS' and gv[1]=='Cross' and gv[2]=='De': continue
        xvar = "GABA"
        yvar = "value"
        g = sns.lmplot(data=gr, 
                  row='Presentation',col='measure',# facet rows and columns
                  x=xvar, y=yvar,hue="Population",sharey=False,ci=False)
        g.fig.suptitle(gv, fontsize=16, y=0.97)
        g.fig.subplots_adjust(top=.9, right=.8)
        g.set_axis_labels(xvar, yvar)
        pdf.savefig(g.fig)
    plt.close('all')

('OS', 'Cross', 'De') False
('OS', 'Cross', 'Nde') False
('OS', 'Iso', 'De') False
('OS', 'Iso', 'Nde') False
('SS', 'Cross', 'De') False
('SS', 'Cross', 'Nde') False
('SS', 'Iso', 'De') False
('SS', 'Iso', 'Nde') False


## Model the data again, this time log-transforming the ThreshElev (and not ignoring the facilitation data points)

In [34]:
# initialize parameters
lm_params = s.lf.Parameters()
lm_params.add('y_int', value=1)
lm_params.add('slope', value=1)
lm_params.keys()

odict_keys(['y_int', 'slope'])

In [35]:
# model each group's data using functions defined in suppression.py
preds_log = groups.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh, lm_params,
                     ret='preds', supp_only=False, log=True)

('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De')
Name      Value      Min      Max   Stderr     Vary     Expr
slope  0.005431     -inf      inf 0.009842     True     None
y_int  -0.03112     -inf      inf   0.1037     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'Nde', 'Amblyope-Nde')
Name      Value      Min      Max   Stderr     Vary     Expr
slope  0.006193     -inf      inf   0.0106     True     None
y_int   0.01355     -inf      inf   0.1366     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De')
Name      Value      Min      Max   Stderr     Vary     Expr
slope  0.009282     -inf      inf 0.007325     True     None
y_int  -0.03811     -inf      inf  0.05004     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde')
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.04466     -inf      inf  0.04908     True     None
y_int     0.544     -inf      inf   0.5438     True     None
('OS

In [38]:
preds_log.head(n=150)

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev,ThreshPred
0,ai,SS,nMono,Iso,Nde,Control,10.0,0.615,0.009,1.449,Control-Nde,6.900,-0.211125,-0.262647
1,ai,SS,nMono,Iso,Nde,Control,30.0,0.671,0.010,4.348,Control-Nde,6.900,-0.173277,-0.089551
2,ai,SS,nMono,Iso,Nde,Control,62.0,1.658,0.007,8.986,Control-Nde,6.900,0.219585,0.187380
14,cr,SS,nMono,Iso,Nde,Control,10.0,0.906,0.006,3.201,Control-Nde,3.124,-0.042872,0.022280
15,cr,SS,nMono,Iso,Nde,Control,30.0,2.218,0.005,9.604,Control-Nde,3.124,0.345962,0.254215
16,cr,SS,nMono,Iso,Nde,Control,79.0,6.249,0.006,25.290,Control-Nde,3.124,0.795811,0.822405
17,ct,SS,nMono,Iso,Nde,Control,10.0,0.620,0.007,1.137,Control-Nde,8.794,-0.207608,-0.182410
18,ct,SS,nMono,Iso,Nde,Control,30.0,1.179,0.006,3.411,Control-Nde,8.794,0.071514,0.030568
19,ct,SS,nMono,Iso,Nde,Control,62.0,2.268,0.007,7.050,Control-Nde,8.794,0.355643,0.371390
20,dl,SS,nMono,Iso,Nde,Control,5.0,0.979,0.006,0.866,Control-Nde,5.772,-0.009217,-0.025154


In [46]:
preds_log[(preds_log.Task=="SS") & (preds_log.Orientation=="Cross")]

,Subject,Task,Presentation,Orientation,Eye,Population,MaskContrast,ThreshElev,ThreshElev_SE,RelMaskContrast,Trace,BaselineThresh,logThreshElev,ThreshPred
246,ai,SS,nMono,Cross,Nde,Control,10.0,0.480,0.008,1.449,Control-Nde,6.900,-0.318759,-0.296355
247,ai,SS,nMono,Cross,Nde,Control,30.0,0.698,0.008,4.348,Control-Nde,6.900,-0.156145,-0.192551
248,ai,SS,nMono,Cross,Nde,Control,62.0,0.911,0.009,8.986,Control-Nde,6.900,-0.040482,-0.026478
260,cr,SS,nMono,Cross,Nde,Control,10.0,0.976,0.007,3.201,Control-Nde,3.124,-0.010550,-0.086399
261,cr,SS,nMono,Cross,Nde,Control,30.0,0.814,0.005,9.604,Control-Nde,3.124,-0.089376,0.017434
262,cr,SS,nMono,Cross,Nde,Control,79.0,2.008,0.005,25.290,Control-Nde,3.124,0.302764,0.271802
263,ct,SS,nMono,Cross,Nde,Control,10.0,0.884,0.007,1.137,Control-Nde,8.794,-0.053548,-0.119381
264,ct,SS,nMono,Cross,Nde,Control,30.0,0.784,0.011,3.411,Control-Nde,8.794,-0.105684,0.001288
265,ct,SS,nMono,Cross,Nde,Control,62.0,1.720,0.007,7.050,Control-Nde,8.794,0.235528,0.194390
266,dl,SS,nMono,Cross,Nde,Control,5.0,1.251,0.008,0.866,Control-Nde,5.772,0.097257,0.007046


In [47]:
s.group_facet_plots(preds_log, s.fit_plot, 'plots/combined_loglinear_logaxes_withpred.pdf',
                  ['Task','Orientation'], #each combo of this gets its own page
                  row='Presentation',col='Trace',# facet rows and columns
                  x="RelMaskContrast", y="logThreshElev", # x, y
                  hue="Subject",yerr='ThreshElev_SE',fmt_obs='o',fmt_pred='.--',Ycol="ThreshPred") 

Plotting OS.Cross
Plotting OS.Iso
Plotting SS.Cross
Plotting SS.Iso
Plots saved at plots/combined_loglinear_logaxes_withpred.pdf


In [159]:
# model each group's data using functions defined in suppression.py, return params
pfit_log = groups_with_baseline.apply(s.model_condition, s.linear_nofac_err, s.linear_nofac_thresh, lm_params, ret='weights', supp_only=False, log=True).reset_index()

('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'De', 'Amblyope-De', 2.4859999999999998)
Name      Value      Min      Max   Stderr     Vary     Expr
slope  0.005431     -inf      inf 0.009842     True     None
y_int  -0.03112     -inf      inf   0.1037     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'am', 'Nde', 'Amblyope-Nde', 2.7539999999999996)
Name      Value      Min      Max   Stderr     Vary     Expr
slope  0.006193     -inf      inf   0.0106     True     None
y_int   0.01355     -inf      inf   0.1366     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'De', 'Amblyope-De', 1.76)
Name      Value      Min      Max   Stderr     Vary     Expr
slope  0.009282     -inf      inf 0.007325     True     None
y_int  -0.03811     -inf      inf  0.05004     True     None
('OS', 'Cross', 'nDicho', 'Amblyope', 'bi', 'Nde', 'Amblyope-Nde', 1.9159999999999999)
Name      Value      Min      Max   Stderr     Vary     Expr
slope   0.04466     -inf      inf  0.04908     True     None

In [160]:
pfit_log

,Task,Orientation,Presentation,Population,Subject,Eye,Trace,BaselineThresh,y_int,slope
0,OS,Cross,nDicho,Amblyope,am,De,Amblyope-De,2.486,-0.031123,0.005431
1,OS,Cross,nDicho,Amblyope,am,Nde,Amblyope-Nde,2.754,0.013548,0.006193
2,OS,Cross,nDicho,Amblyope,bi,De,Amblyope-De,1.760,-0.038109,0.009282
3,OS,Cross,nDicho,Amblyope,bi,Nde,Amblyope-Nde,1.916,0.543966,0.044659
4,OS,Cross,nDicho,Amblyope,gd,De,Amblyope-De,2.071,-0.083783,0.018723
5,OS,Cross,nDicho,Amblyope,gd,Nde,Amblyope-Nde,2.112,0.212457,0.032965
6,OS,Cross,nDicho,Amblyope,jv,De,Amblyope-De,1.777,-0.155366,0.024072
7,OS,Cross,nDicho,Amblyope,jv,Nde,Amblyope-Nde,2.482,0.272295,0.015633
8,OS,Cross,nDicho,Amblyope,li,De,Amblyope-De,2.238,0.132879,-0.008719
9,OS,Cross,nDicho,Amblyope,li,Nde,Amblyope-Nde,4.555,0.625618,0.007074


In [162]:
# melt the result of the modeling into long format for plotting
m = pd.melt(pfit_log, id_vars=pp_gvars,var_name='measure')
m

,Task,Orientation,Presentation,Population,Subject,Eye,Trace,measure,value
0,OS,Cross,nDicho,Amblyope,am,De,Amblyope-De,BaselineThresh,2.486000
1,OS,Cross,nDicho,Amblyope,am,Nde,Amblyope-Nde,BaselineThresh,2.754000
2,OS,Cross,nDicho,Amblyope,bi,De,Amblyope-De,BaselineThresh,1.760000
3,OS,Cross,nDicho,Amblyope,bi,Nde,Amblyope-Nde,BaselineThresh,1.916000
4,OS,Cross,nDicho,Amblyope,gd,De,Amblyope-De,BaselineThresh,2.071000
5,OS,Cross,nDicho,Amblyope,gd,Nde,Amblyope-Nde,BaselineThresh,2.112000
6,OS,Cross,nDicho,Amblyope,jv,De,Amblyope-De,BaselineThresh,1.777000
7,OS,Cross,nDicho,Amblyope,jv,Nde,Amblyope-Nde,BaselineThresh,2.482000
8,OS,Cross,nDicho,Amblyope,li,De,Amblyope-De,BaselineThresh,2.238000
9,OS,Cross,nDicho,Amblyope,li,Nde,Amblyope-Nde,BaselineThresh,4.555000


In [163]:
#Grab the GABA measure for each subject and it to each observation for easy plotting

comb = m.join(gdf.set_index(['subjName'])['GABA'], on=['Subject'])

In [164]:
#graphs!
with s.PdfPages('plots/gaba_vs_measures_loglinear.pdf') as pdf:
    plot_groups = comb.groupby(['Task','Orientation','Eye'])
    for gv, gr in plot_groups:
        print(gv, np.all(np.isnan(gr['value'])))
        if gv[0]=='SS' and gv[1]=='Cross' and gv[2]=='De': continue
        xvar = "GABA"
        yvar = "value"
        g = sns.lmplot(data=gr, 
                  row='Presentation',col='measure',# facet rows and columns
                  x=xvar, y=yvar,hue="Population",sharey=False,ci=False)
        g.fig.suptitle(gv, fontsize=16, y=0.97)
        g.fig.subplots_adjust(top=.9, right=.8)
        g.set_axis_labels(xvar, yvar)
        pdf.savefig(g.fig)
    plt.close('all')

('OS', 'Cross', 'De') False
('OS', 'Cross', 'Nde') False
('OS', 'Iso', 'De') False
('OS', 'Iso', 'Nde') False
('SS', 'Cross', 'De') False
('SS', 'Cross', 'Nde') False
('SS', 'Iso', 'De') False
('SS', 'Iso', 'Nde') False


### Old Stuff ###

In [ ]:
#model each group's data and return a dataframe
# model_subj is a function defined in suppression.py
lm = groups.apply(model_subj).reset_index() # reset index makes the grouping vars back into columns

In [ ]:
# melt the result of the modeling into long format for plotting
m = pd.melt(lm, id_vars=pp_gvars,var_name='measure')
m.head()

In [ ]:
m['gaba_val'] = m['subjName'].map(gdf.set_index('subjName')['GABA'])
m.head()

In [ ]:
mslope = m[m['measure']=="slope_lm"] # get only the slope data

In [ ]:
group_facet_plots(mslope, sns.regplot, 'plots/reg_scatter.pdf',
                  ['Task','measure','Presentation','Orientation'], #each combo of this gets its own page
                  col='Eye', row=None,
                  x="gaba_val", y="value", hue="Population") #arguments to be passed to the plotting function